In [1]:
import optuna
from utils import *
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [3]:
study_name = "SLIMBPR"
study = optuna.create_study(
    study_name=study_name,
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-07 21:07:36,547] Using an existing study with name 'SLIMBPR' instead of creating a new one.


In [4]:
def objective(trial):
    epochs = trial.suggest_int('epochs', 1, 500, log=True)
    symmetric = trial.suggest_categorical('symmetric', [True, False])
    lambda_i = trial.suggest_float('lambda_i', 1e-8, 1, log=True)
    lambda_j = trial.suggest_float('lambda_j', 1e-8, 1, log=True)
    learning_rate = trial.suggest_float('learning_rate', 1e-8, 1, log=True)
    topK = trial.suggest_int('topK', 1, 1000, log=True)
    sgd_mode = trial.suggest_categorical('sgd_mode', ["sgd", "adagrad", "adam"])
    gamma = trial.suggest_float('gamma', 1e-8, 1, log=True)
    beta_1 = trial.suggest_float('beta_1', 1e-8, 1, log=True)
    beta_2 = trial.suggest_float('beta_2', 1e-8, 1, log=True)
    
    recommender = SLIM_BPR_Cython(data_train, verbose=False)
    recommender.fit(
        epochs=epochs,
        symmetric=symmetric,
        lambda_i=lambda_i,
        lambda_j=lambda_j,
        learning_rate=learning_rate,
        topK=topK,
        sgd_mode=sgd_mode,
        gamma=gamma,
        beta_1=beta_1,
        beta_2=beta_2,
    )
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=300)

SLIM_BPR_Recommender: Epoch 1 of 450. Elapsed time 0.05 sec
SLIM_BPR_Recommender: Epoch 2 of 450. Elapsed time 0.09 sec
SLIM_BPR_Recommender: Epoch 3 of 450. Elapsed time 0.14 sec
SLIM_BPR_Recommender: Epoch 4 of 450. Elapsed time 0.18 sec
SLIM_BPR_Recommender: Epoch 5 of 450. Elapsed time 0.22 sec
SLIM_BPR_Recommender: Epoch 6 of 450. Elapsed time 0.27 sec
SLIM_BPR_Recommender: Epoch 7 of 450. Elapsed time 0.31 sec
SLIM_BPR_Recommender: Epoch 8 of 450. Elapsed time 0.36 sec
SLIM_BPR_Recommender: Epoch 9 of 450. Elapsed time 0.40 sec
SLIM_BPR_Recommender: Epoch 10 of 450. Elapsed time 0.45 sec
SLIM_BPR_Recommender: Epoch 11 of 450. Elapsed time 0.49 sec
SLIM_BPR_Recommender: Epoch 12 of 450. Elapsed time 0.54 sec
SLIM_BPR_Recommender: Epoch 13 of 450. Elapsed time 0.58 sec
SLIM_BPR_Recommender: Epoch 14 of 450. Elapsed time 0.62 sec
SLIM_BPR_Recommender: Epoch 15 of 450. Elapsed time 0.67 sec
SLIM_BPR_Recommender: Epoch 16 of 450. Elapsed time 0.71 sec
SLIM_BPR_Recommender: Epoch 17 of

[W 2024-01-07 21:08:34,328] Trial 300 failed with parameters: {'epochs': 450, 'symmetric': True, 'lambda_i': 0.46386315650310234, 'lambda_j': 0.008153493034401429, 'learning_rate': 0.00013546307023034603, 'topK': 80, 'sgd_mode': 'adam', 'gamma': 5.010552445742985e-06, 'beta_1': 3.11931245994755e-07, 'beta_2': 0.0007996907661681623} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/kinami/.conda/envs/RecSysFramework/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_108475/3317402386.py", line 14, in objective
    recommender.fit(
  File "/home/kinami/code/rcs/Recommenders/SLIM/Cython/SLIM_BPR_Cython.py", line 158, in fit
    self.get_S_incremental_and_set_W()
  File "/home/kinami/code/rcs/Recommenders/SLIM/Cython/SLIM_BPR_Cython.py", line 180, in get_S_incremental_and_set_W
    self.S_incremental = self.cythonEpoch.get_S()
  File "SLIM_BPR_Cython_Epo

KeyboardInterrupt: 